In [41]:
from honeybee_energy.schedule.day import ScheduleDay
import honeybee_energy.lib.scheduletypelimits as schedule_types
from ladybug.dt import Time, Date
from ladybug.datacollection import HourlyContinuousCollection
from ladybug.analysisperiod import AnalysisPeriod
from ladybug.datatype.generic import GenericType

simple_office = ScheduleDay('Simple Office Occupancy',[0, 1, 0], [Time(0, 0), Time(9, 0), Time(17, 0)])

print(list(simple_office.data_collection().values))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
from honeybee_energy.schedule.day import ScheduleDay
from honeybee_energy.schedule.rule import ScheduleRule
from honeybee_energy.schedule.ruleset import ScheduleRuleset
from honeybee_energy.schedule.typelimit import ScheduleTypeLimit
import honeybee_energy.lib.scheduletypelimits as schedule_types

from ladybug.dt import Date, Time
from ladybug.datatype import fraction
from ladybug.analysisperiod import AnalysisPeriod

weekday_office = ScheduleDay('Weekday Office Occupancy', [0, 1, 0],
                                 [Time(0, 0), Time(9, 0), Time(17, 0)])
saturday_office = ScheduleDay('Saturday Office Occupancy', [0, 0.25, 0],
                                [Time(0, 0), Time(9, 0), Time(17, 0)])
sunday_office = ScheduleDay('Sunday Office Occupancy', [0])
sat_rule = ScheduleRule(saturday_office, apply_saturday=True)
sun_rule = ScheduleRule(sunday_office, apply_sunday=True)
summer_office = ScheduleDay('Summer Office Occupancy', [0, 1, 0.25],
                            [Time(0, 0), Time(6, 0), Time(22, 0)])
winter_office = ScheduleDay('Winter Office Occupancy', [0])
schedule = ScheduleRuleset('Office Occupancy', weekday_office,
                            [sat_rule, sun_rule], schedule_types.fractional,
                            sunday_office, summer_office, winter_office)

schedule.schedule_rules

(ScheduleRule: Saturday Office Occupancy [days applied: saturday] [date range: 01 Jan - 31 Dec],
 ScheduleRule: Sunday Office Occupancy [days applied: sunday] [date range: 01 Jan - 31 Dec])

In [22]:
room.properties.energy.lighting.schedule[0]

ScheduleRule: Hospital BLDG_LIGHT_EXTD_SCH_Sat [days applied: saturday] [date range: 01 Jan - 31 Dec]

In [2]:
print(room.properties.energy.lighting.schedule.summer_designday_schedule)

Schedule:Day:Interval,
 Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn, !- schedule name
 ,                         !- schedule type limits
 No,                       !- interpolate to timestep
 24:00,                    !- time 1 {hh:mm}
 1.0;                      !- value until time 1


In [2]:
from honeybee.model import Model
from honeybee_doe2.properties.activitydescription import DayScheduleDoe, DayScheduleType, WeekScheduleDoe


tmodel = Model.from_file('tests\\assets\\hospcube.hbjson')
room1 = tmodel.rooms[0]


my_sch = WeekScheduleDoe.from_schedule_ruleset(room1.properties.energy.lighting.schedule, stype=DayScheduleType.FRACTION)
my_sch

"Hospital BLDG_LIGHT_EXTD_SCH" = WEEK-SCHEDULE
   TYPE     = FRACTION
     (CDD)  "Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn"
     (HDD)  "Hospital BLDG_LIGHT_EXTD_SCH_WntrDsn"
     (HOL)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
     (SAT)  "Hospital BLDG_LIGHT_EXTD_SCH_Sat"
     (MON, FRI)  "Hospital BLDG_LIGHT_EXTD_SCH_Wkdy"
     (SUN)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
   ..

In [1]:
from honeybee.model import Model
from honeybee_doe2.properties.activitydescription import DayScheduleDoe, DayScheduleType, WeekScheduleDoe


tmodel = Model.from_file('tests\\assets\\hospcube.hbjson')
room1 = tmodel.rooms[0]


my_sch = WeekScheduleDoe.from_schedule_ruleset(room1.properties.energy.lighting.schedule, stype=DayScheduleType.FRACTION)
my_sch

"Hospital BLDG_LIGHT_EXTD_SCH" = WEEK-SCHEDULE
   TYPE     = FRACTION
     (CDD)  "Hospital BLDG_LIGHT_EXTD_SCH_SmrDsn"
     (HDD)  "Hospital BLDG_LIGHT_EXTD_SCH_WntrDsn"
     (HOL)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
     (SAT)  "Hospital BLDG_LIGHT_EXTD_SCH_Sat"
     (MON, FRI)  "Hospital BLDG_LIGHT_EXTD_SCH_Wkdy"
     (SUN)  "Hospital BLDG_LIGHT_EXTD_SCH_Default"
   ..